# Import and Initialization

In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf

# standard library imports
from __future__ import absolute_import, division, print_function

# standard numerical library imports
import numpy as np

# energyflow imports
import energyflow as ef
from energyflow.archs import PFN
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, remap_pids, to_categorical

from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt


import pandas
import h5py
import pickle

import datetime

2022-05-03 15:52:59.338720: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# configs
train, val, test = 0.6, 0.3, 0.1
Phi_sizes, F_sizes = (256, 256, 256), (256, 256, 256)
num_epoch = 500
batch_size = 1000

# Load and Process Data

In [3]:
signal_1_df = "/global/home/users/yifengh3/VAE/new_data/muon_data/h_signal_big_muons.h5"
signal_2_df = "/global/home/users/yifengh3/VAE/new_data/muon_data/hv_signal_big_muons.h5"
raw_signal_1 = pandas.read_hdf(signal_1_df)
raw_signal_2 = pandas.read_hdf(signal_2_df)

In [4]:
print("signal_1 data shape: {}".format(raw_signal_1.shape))
print("signal_2 data shape: {}".format(raw_signal_2.shape))

signal_1 data shape: (173692, 200)
signal_2 data shape: (157772, 200)


In [21]:
raw_signal_1

,pT0,eta0,phi0,E0,pT1,eta1,phi1,E1,pT2,eta2,...,phi47,E47,pT48,eta48,phi48,E48,pT49,eta49,phi49,E49
0,-13.0,40.94240,-1.718690,40.94240,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-13.0,33.92570,-0.287077,33.92570,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-13.0,4.71550,-0.290955,4.71550,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-13.0,12.91650,-1.806460,12.91650,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-13.0,16.97240,-1.327890,16.97240,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6881,-13.0,20.39380,-2.300500,20.39380,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6882,-13.0,8.02589,-1.974860,8.02589,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6883,-13.0,59.43760,0.043301,59.43760,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6884,-13.0,36.42490,1.648680,36.42490,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Truncate the nan
signal_1 = raw_signal_1.to_numpy()[:,:8]
signal_2 = raw_signal_2.to_numpy()[:,:8]

In [31]:
np.mean(signal_1,axis=0)

array([-13.0, 32.752112992999315, 0.001057640618451052,
       32.752112992999315, 0.0, 0.0, 0.0, 0.0], dtype=object)

In [6]:
print("signal_1 data shape: {}".format(signal_1.shape))
print("signal_2 data shape: {}".format(signal_2.shape))

signal_1 data shape: (173692, 8)
signal_2 data shape: (157772, 8)


In [7]:
# assign labels to signal and background data, 0 for sig1,  1 for sig2
# (updated since we might get multiple signals) 
labeled_sig1 = np.append(signal_1,np.zeros((signal_1.shape[0],1)),axis=1)
labeled_sig2 = np.append(signal_2,np.ones((signal_2.shape[0],1)),axis=1)

In [8]:
# mix two data array into one signal array
data = np.concatenate((labeled_sig1,labeled_sig2))

#and shuffle the data
np.random.shuffle(data)

In [9]:
X = data[:,:-1]
y = data[:,-1]

In [10]:
print("shape of X: {}".format(X.shape))
print("shape of Y: {}".format(y.shape))

shape of X: (331464, 8)
shape of Y: (331464,)


In [11]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
total = labeled_sig1.shape[0] + labeled_sig2.shape[0]
weight_for_0 = (1 / labeled_sig1.shape[0]) * (total / 2.0)
weight_for_1 = (1 / labeled_sig2.shape[0]) * (total / 2.0)


class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for background: {:.2f}'.format(weight_for_0))
print('Weight for signal: {:.2f}'.format(weight_for_1))

Weight for background: 0.95
Weight for signal: 1.05


In [12]:
# To categorical as stipulated in example
Y = to_categorical(y, num_classes=2)

# Reshape X to shape (number of jets, 50, 4)
X = X.reshape(-1,2,4)

# ignore the pid info
X = X[:,:,:3]

In [13]:
print(X.shape)

(331464, 2, 3)


In [14]:
# # normalizing jets
# # copied from example
# import tqdm
# for x in tqdm.tqdm(X):
#     # now add the status bar :)
#     mask = x[:,0] > 0
#     yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)
#     x[mask,1:3] -= yphi_avg
#     x[mask,0] /= x[:,0].sum()

In [15]:
print('Finished preprocessing')
print("shape of X: {}".format(X.shape))
print("shape of Y: {}".format(y.shape))

Finished preprocessing
shape of X: (331464, 2, 3)
shape of Y: (331464,)


In [16]:
X = X.astype('float64')

In [17]:
# do train/val/test split 
(X_train, X_val, X_test,
 Y_train, Y_val, Y_test) = data_split(X, Y, val=val, test=test)

# Build and Train the Model

In [18]:
print('Model summary:')

# build architecture
pfn = PFN(input_dim=X.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes)

Model summary:


2022-05-03 15:53:40.451290: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-03 15:53:40.452453: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-03 15:53:40.481862: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:81:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-05-03 15:53:40.481884: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-03 15:53:40.483388: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-05-03 15:53:40.483418: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-0

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
tdist_0 (TimeDistributed)       (None, None, 256)    1024        input[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, None, 256)    0           tdist_0[0][0]                    
__________________________________________________________________________________________________
tdist_1 (TimeDistributed)       (None, None, 256)    65792       activation[0][0]                 
______________________________________________________________________________________________

In [19]:
# now train the model

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.1**(1/5), patience=5, min_lr=1e-5,
                                                verbose=1)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, 
                                              verbose=1)

callbacks = [reduce_lr,early_stop]

hist1 = pfn.fit(X_train, Y_train,
        epochs=num_epoch,
        batch_size=batch_size,
        validation_data=(X_val, Y_val),
        class_weight=class_weight,
        callbacks=callbacks,
        verbose=1)

Epoch 1/500


2022-05-03 15:53:41.319799: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-03 15:53:41.320266: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2994530000 Hz


  1/199 [..............................] - ETA: 1:49 - loss: 10.0800 - acc: 0.5320

2022-05-03 15:53:41.715932: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


199/199 [==============================] - 2s 5ms/step - loss: 3.0631 - acc: 0.6979 - val_loss: 0.4942 - val_acc: 0.7704
Epoch 2/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4983 - acc: 0.7703 - val_loss: 0.4813 - val_acc: 0.7779
Epoch 3/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4895 - acc: 0.7718 - val_loss: 0.4801 - val_acc: 0.7789
Epoch 4/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4885 - acc: 0.7715 - val_loss: 0.4822 - val_acc: 0.7789
Epoch 5/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4866 - acc: 0.7730 - val_loss: 0.4813 - val_acc: 0.7779
Epoch 6/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4893 - acc: 0.7723 - val_loss: 0.4792 - val_acc: 0.7790
Epoch 7/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4852 - acc: 0.7746 - val_loss: 0.4814 - val_acc: 0.7782
Epoch 8/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4866

Epoch 57/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4829 - acc: 0.7750 - val_loss: 0.4793 - val_acc: 0.7787
Epoch 58/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4819 - acc: 0.7758 - val_loss: 0.4787 - val_acc: 0.7787
Epoch 59/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4814 - acc: 0.7755 - val_loss: 0.4792 - val_acc: 0.7784
Epoch 60/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4824 - acc: 0.7753 - val_loss: 0.4792 - val_acc: 0.7786
Epoch 61/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4834 - acc: 0.7738 - val_loss: 0.4792 - val_acc: 0.7785
Epoch 62/500
199/199 [==============================] - 1s 4ms/step - loss: 0.4833 - acc: 0.7740 - val_loss: 0.4800 - val_acc: 0.7781
Epoch 63/500
199/199 [==============================] - 1s 3ms/step - loss: 0.4816 - acc: 0.7751 - val_loss: 0.4797 - val_acc: 0.7781

Epoch 00063: ReduceLROnPlateau reducing learning rate to 1.00

# Analyze the Model

In [20]:
# get predictions on test data
preds = pfn.predict(X_test, batch_size=10000)

# get ROC curve
pfn_fp, pfn_tp, threshs = roc_curve(Y_test[:,1], preds[:,1])

# get area under the ROC curve
auc = roc_auc_score(Y_test[:,1], preds[:,1])
print()
print('PFN AUC:', auc)
print()


PFN AUC: 0.8387118129529184



In [ ]:
# get multiplicity and mass for comparison
masses = np.asarray([ef.ms_from_p4s(ef.p4s_from_ptyphims(x).sum(axis=0)) for x in X])
mults = np.asarray([np.count_nonzero(x[:,0]) for x in X])
mass_fp, mass_tp, threshs = roc_curve(Y[:,1], -masses)
mult_fp, mult_tp, threshs = roc_curve(Y[:,1], -mults)

# some nicer plot settings 
plt.rcParams['figure.figsize'] = (4,4)
plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.autolayout'] = True

# plot the ROC curves
plt.plot(pfn_tp, 1-pfn_fp, '-', color='black', label='PFN')
plt.plot(mass_tp, 1-mass_fp, '-', color='blue', label='Jet Mass')
plt.plot(mult_tp, 1-mult_fp, '-', color='red', label='Multiplicity')

# axes labels
plt.xlabel('Quark Jet Efficiency')
plt.ylabel('Gluon Jet Rejection')

# axes limits
plt.xlim(0, 1)
plt.ylim(0, 1)

# make legend and show plot
plt.legend(loc='lower left', frameon=False)
plt.show()


In [ ]:
plt.plot(hist1.history["loss"],label="loss")
plt.plot(hist1.history["val_loss"],label="validation loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(hist1.history["acc"],label="acc")
plt.plot(hist1.history["val_acc"],label="validation acc")
plt.legend()
plt.show()